In [1]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [3]:
dane = pd.read_csv('00_dane/dane_TM.csv',  sep='|', encoding='UTF-8')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x.lower()))
nlp = spacy.load('pl_model')
stop = set(nlp.Defaults.stop_words)
stop.add('zł')
stop.add('pkt')
stop.add('art')
stop.add('ustawy')
stop.add('r')
dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split() if word not in stop])

In [4]:
dane.head(3)

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,nr_tematu_LDA,opis_tematu_LDA,nr_tematu_LDA_TF,opis_tematu_LDA_TF
0,0,01-02-1992,Chciałbym kilka zdań powiedzieć na temat skut...,Aleksander Bentkowski,Polskie Stronnictwo Ludowe,"[chcieć, zdanie, powiedzieć, temat, skutek, us...",30.0,przestępczość,10.0,sądy
1,1,01-02-1992,"Przysłuchując się tej bardzo emocjonalnej, dł...",Aleksander Kwaśniewski,Sojusz Lewicy Demokratycznej,"[przysłuchiwać, emocjonalny, długo, trwać, deb...",10.0,polityka,20.0,polityka
2,2,01-02-1992,"Różne przebyliśmy drogi, żeby tu dzisiaj powi...",Alojzy Pietrzyk,Klub Parlamentarny NSZZ Solidarność,"[różny, przebyć, droga, powiedzieć, swój, ocze...",2.0,NaN,20.0,polityka


Utworzenie słownika autor - dokument

In [14]:
aut2doc=dict()
for (aut, doc_nr) in zip(dane['full_name'], dane['nr']):
    if aut not in aut2doc.keys():
        aut2doc[aut]=[]
    aut2doc[aut].append(doc_nr)

In [17]:
aut2doc['Janusz Korwin-Mikke']

[996,
 997,
 998,
 1379,
 1380,
 2154,
 2155,
 4046,
 4047,
 4656,
 4657,
 4658,
 4659,
 5133,
 6148,
 6149,
 6620,
 6621,
 7339,
 7390,
 8486,
 8487,
 8662,
 8663,
 10056,
 10057,
 10280,
 10557,
 11844,
 11845,
 12935,
 12936,
 14928,
 14929,
 15381,
 15382,
 15453,
 15454,
 18338,
 18996,
 20692,
 20693,
 21308,
 21367,
 21995,
 21996,
 22389,
 22390,
 24580,
 24581,
 24582,
 24583,
 26094,
 27458,
 27459,
 29797,
 29798,
 29799,
 30154,
 31280,
 31281,
 31282,
 31283,
 32914,
 32915,
 35708,
 35709,
 35710,
 37321,
 39230,
 39231,
 39232,
 39233,
 43065,
 46908,
 49073,
 49957,
 49958,
 53852,
 53853,
 55814,
 60277,
 61560,
 65579,
 67265,
 72926,
 72927,
 73197,
 73198,
 73199,
 74338,
 74339,
 77502,
 77503,
 78409,
 78410,
 78411,
 79286,
 79287,
 79288,
 79289,
 81882,
 81883,
 81884,
 82941,
 83147,
 83148,
 83914,
 83915,
 83979,
 83980,
 83981,
 85054,
 85791,
 85792,
 88129,
 89176,
 89177,
 89252,
 89253,
 90241,
 90242,
 90243,
 91093,
 91094,
 91171,
 91172,
 92441,
 92

In [28]:
doc2aut = dane['full_name'].T.to_dict()

In [31]:
doc2aut[122846]

'Janusz Korwin-Mikke'

Przygotowanie słowników itp.

In [32]:
corp = dane['speech_rawest']
bigram = gensim.models.Phrases(corp, min_count=20, threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
corp = [bigram_mod[doc] for doc in corp]

In [33]:
dictionary = gensim.corpora.Dictionary(corp)
dictionary.filter_extremes(no_above=0.3, no_below=50)


In [34]:
BOW = [dictionary.doc2bow(text) for text in corp]

In [35]:
from gensim.models import AuthorTopicModel
%time model = AuthorTopicModel(corpus=BOW, num_topics=30, id2word=dictionary.id2token, \
                author2doc=aut2doc, passes=5, eval_every=0, \
                iterations=1, random_state=1)

ValueError: cannot compute the author-topic model over an empty collection (no terms)